In [5]:
import sys
sys.path.append("../src")

import torch
import json
import numpy as np
import loading_json as load
import json
import generate_kvadraturen as gk

In [6]:
# Load results from optimization
f = open("../optimization_results/kvadraturen_optimization/network22_config22_fwd.json")
results = json.load(f)
f.close()

In [10]:
# Collecting network configuration
network_file = results['network_file']
config_file = results['config_file']
f = open("../"+network_file)
network_config = json.load(f)
f.close()

T = network_config['T']
N = network_config['N']
controls = network_config['control_points']

In [12]:
n_speeds = []
last_speed_idx = 0
n_cycles = []
control_points = []
config = None

def update_nspeeds_ncycles_controls(speed_limits, cycle_times, new_control_points):
    global n_speeds
    global n_cycles
    global control_points
    global last_speed_idx

    n_speeds = []
    n_cycles = []
    speed_idx = 0
    for speeds in speed_limits:
        n_speeds.append(len(speeds))
        speed_idx += len(speeds)
    last_speed_idx = speed_idx

    for cycles in cycle_times:
        n_cycles.append(len(cycles))

    control_points = new_control_points

def update_config(config_data):
    global config
    config = config_data

def load_bus_network(network_file, config_file):
    '''
    Function for initializing a bus network modelling kvadraturen
    with initial speed limits and speed limits as specified in the file
    filename. The grid spacing is also specified in the file
    '''
    f = open("../" + network_file)
    data = json.load(f)
    f.close()
    T = data["T"]
    N = data["N"]
    speed_limits = data["speed_limits"] # Nested list
    control_points = data["control_points"] # Nested list
    cycle_times = data["cycle_times"] # Nested list

    update_nspeeds_ncycles_controls(speed_limits, cycle_times, control_points)

    f = open("../" + config_file)
    data = json.load(f)
    f.close()
    update_config(data)
    
    return T, N, speed_limits, cycle_times

def get_speeds_cycles_from_params(params):
    idx = 0
    speed_limits = []
    cycle_times = []

    for i in range(len(n_speeds)):
        speed_limits.append([])
        for j in range(n_speeds[i]):
            speed_limits[i].append(params[idx])
            idx += 1

    for i in range(len(n_cycles)):
        cycle_times.append([])
        for j in range(n_cycles[i]):
            cycle_times[i].append(params[idx])
            idx += 1

    return speed_limits, cycle_times
def create_network_from_params(T, N, params, track_grad = False):
    speed_limits, cycle_times = get_speeds_cycles_from_params(params)
    # bus_network = gk.generate_kvadraturen_roundabout_w_params(T, N, speed_limits, control_points, cycle_times,
    #                                                           track_grad=track_grad)
    bus_network = gk.generate_kvadraturen_from_config_e18(T, N, speed_limits, control_points,
                                                          cycle_times, config, track_grad=track_grad)
    return bus_network

In [13]:
# Collecting the start and final parameters
start = results['parameters'][0]
opt = results['parameters'][-1]

In [14]:
T, N, speed_limits, cycle_times = load_bus_network(network_file, config_file)

In [16]:
start_network = create_network_from_params(T, N, start, track_grad=False)
opt_network = create_network_from_params(T, N, opt, track_grad=False)

In [18]:
# Do the simulations
densities, _, lengths, delays, n_stops = start_network.solve_cons_law_counting()
opt_densities, _, opt_lengths, opt_delays, opt_n_stops = opt_network.solve_cons_law_counting()

Bus 19_bw1 reached bus stop 0 at time 28.75, should wait for 29.854570388793945 seconds
Bus 19_fw1 reached bus stop 0 at time 44.125, should wait for 29.976776123046875 seconds
Bus m1_bw1 reached bus stop 0 at time 106.6875, should wait for 29.81698226928711 seconds
Bus custom_bw1 reached bus stop 0 at time 121.875, should wait for 29.89146614074707 seconds
Bus 19_bw1 reached bus stop 1 at time 143.9375, should wait for 29.88227081298828 seconds
Bus m1_bw1 reached bus stop 1 at time 156.125, should wait for 29.86902618408203 seconds
Bus 19_bw1 reached bus stop 2 at time 192.8125, should wait for 29.88826560974121 seconds
Bus m1_fw1 reached bus stop 0 at time 205.75, should wait for 29.812915802001953 seconds
Bus 19_fw1 reached bus stop 1 at time 280.75, should wait for 29.894075393676758 seconds
Bus custom_bw1 reached bus stop 1 at time 383.5, should wait for 29.992538452148438 seconds
Bus 19_bw1 reached bus stop 3 at time 394.1875, should wait for 29.953479766845703 seconds
Bus m1_fw1

In [19]:
# Store densities
densities_loaded = load.convert_from_tensor(densities)
bus_lengths_loaded = load.convert_from_tensor(lengths)

end_densities_loaded = load.convert_from_tensor(opt_densities)
end_bus_lengths_loaded = load.convert_from_tensor(opt_lengths)

In [20]:
with open("../general_densities/kvadraturen_start.json", 'w') as fd:
    fd.write(json.dumps([densities_loaded, bus_lengths_loaded]))

In [21]:
with open("../general_densities/kvadraturen_optimal.json", 'w') as fd:
    fd.write(json.dumps([end_densities_loaded, end_bus_lengths_loaded]))

In [22]:
# Convert to the same time scale:
# Use single_lane_optimal as the goal


f = open("../general_densities/kvadraturen_start.json")
data = json.load(f)
f.close()
orig_densities = data[0]
orig_lengths = data[1]

f = open("../general_densities/kvadraturen_optimal.json")
data_goal = json.load(f)
f.close()
goal_densities = data_goal[0]

In [23]:
goal_times = list(goal_densities['0'].keys())
goal_times = [float(t) for t in goal_times]

old_times = list(orig_densities['0'].keys())
old_times = [float(t) for t in old_times]

In [24]:
new_lengths = {i : {} for i in orig_lengths.keys()}
new_densities = {i : {} for i in orig_densities.keys()}

prev_idx = 0
for t in goal_times:
    add_incr = 0
    for t_prev, t_next in zip(old_times[prev_idx:-1], old_times[prev_idx+1:]):
        if t_prev <= t and t_next >= t:
            prev_dist = t - t_prev
            next_dist = t_next - t
            interval_length = t_next - t_prev
            prev_weight = (interval_length - prev_dist) / interval_length
            next_weight = (interval_length - next_dist) / interval_length
            for bus_id in orig_lengths.keys(): 
                new_lengths[bus_id][str(t)] = orig_lengths[bus_id][str(t_prev)]*prev_weight + orig_lengths[bus_id][str(t_next)]*next_weight
            for road_id in orig_densities.keys():
                new_densities[road_id][str(t)] = [orig_densities[road_id][str(t_prev)][i]*prev_weight + orig_densities[road_id][str(t_next)][i]*next_weight for i in range(len(orig_densities[road_id][str(t_prev)]))]
            break
        else:
            add_incr += 1

In [25]:
# Write to file
with open("../general_densities/kvadraturen_start_opt_times.json", 'w') as fd:
    fd.write(json.dumps([new_densities, new_lengths]))